<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo4/Copia_de_M5U4_Taller_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1VV2e_u46fNm_ewns8QW2HGRZAPHh-e2t" alt = "Encabezado MLDS" width = "100%">  </img>

# **Taller 4 - *Transformers***
---

En este taller usted tendrá que usar un modelo de Transformer de [Hugginface](https://huggingface.co/) para hacer un proceso de *Fine Tunning*. El modelo final será un clasificador de sarcasmo. Usaremos un conjunto de datos que consiste en encabezados de periódicos (textos en inglés).

Importemos las librerías requeridas para solucionar la tarea:

> **Importante:  Recomendamos utilizar GPU para la ejecución de este notebook, ya que puede tomar mucho tiempo la ejeución de algunos casos de prueba en caso de que no se utilice.**

In [4]:
!pip install rlxcrypt

In [5]:
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

In [6]:
import rlxcrypt
import session

grader = session.LoginSequence("DLIAAPCP-GroupMLDS-5-2024-2@4aab2a5b-740f-4fb9-94d4-163161c86a1e")

Please enter your UNCode username: nelson.ferrucho@gmail.com
Please enter your password: ··········


In [7]:
!pip install --upgrade transformers
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [8]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1' # Establece la versión legacy

In [9]:
import numpy as np
import json
import tensorflow as tf
import keras
import transformers
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [10]:
# Versiones de las librerías usadas.
!python --version
print('Numpy', np.__version__)
print('Transformers', transformers.__version__)
print('Tensorflow', tf.__version__)

Python 3.10.12
Numpy 1.26.4
Transformers 4.46.1
Tensorflow 2.17.0


Esta actividad se realizó con las siguientes versiones:
*  Python 3.9.16
*  Numpy 1.22.4
*  Transformers 4.28.1
*  Tensorflow 2.12.0

## **Cargar Datos**
---

El conjunto de datos esta compuesto por titulares de periódicos. Los datos fueron preparados por _Laurence Moroney_ (código fuente disponible en [GitHub](https://github.com/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%203%20-%20Lesson%202c.ipynb)).

In [11]:
!wget -O sarcasm.json https://raw.githubusercontent.com/mindlab-unal/mlds5-dataset-unit4-Tarea4-Transformers/main/sarcasm.json?raw=true
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)
sentences = []
labels = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
print(len(sentences))

--2024-10-31 06:20:09--  https://raw.githubusercontent.com/mindlab-unal/mlds5-dataset-unit4-Tarea4-Transformers/main/sarcasm.json?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616834 (5.4M) [text/plain]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.36M  --.-KB/s    in 0.08s   

2024-10-31 06:20:09 (68.2 MB/s) - ‘sarcasm.json’ saved [5616834/5616834]

26709


Tenemos 26709 muestras. Démosle un vistazo a los datos:

In [12]:
print('label','\t','sentence','\n','----\t----------')
for i in np.random.choice(26709, 10, replace=False):
  print(labels[i],'\t',sentences[i],'\n')

label 	 sentence 
 ----	----------
1 	 9-pound infant barrels way down birth canal 

0 	 facebook to begin letting users know if their data was harvested by cambridge analytica 

0 	 texas man kills co-worker, then takes own life 

1 	 group that makes dodge truck commercials called 'creative team' 

0 	 here's what happens when a spouse who identified as straight comes out as lgbt 

0 	 officials reach deal on trans-pacific partnership 

0 	 using special nails to save roofs — and dollars 

0 	 song and dance for leprosy education 

1 	 christian rock band cleans up hotel room 

1 	 iraq declares partial law 



La etiqueta `0` corresponde a _no sarcasmo_ y la `1` a _sarcasmo_. Es decir, _sarcasmo_ es la clase positiva.

## **Particiones de entrenamiento y prueba**
---
Como siempre, es importante separar una parte de los datos que no serán usados en el entrenamiento para evaluar el desempeño del modelo.

In [13]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(sentences, labels, test_size=0.2, stratify = labels, random_state = 30)
X_train, X_val,  y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify = y_temp, random_state = 30)

Hacemos una codificación _one hot_ de las etiquetas de entrenamiento y validación:

In [14]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

Y contamos cuántos textos hay en cada partición:

In [19]:
len(X_train), len(X_val), len(X_test)

(16025, 5342, 5342)

Tenemos entonces 16025 muestras para entrenamiento, 5342 para validación y la misma cantidad para prueba.

# **DistilBERT**
----
[DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) es un modelo Transformer pequeño, rápido, poco costoso y ligero, entrenado a partir BERT, con una técnica llamada [_Destilación de conocimiento_](https://en.wikipedia.org/wiki/Knowledge_distillation).

La destilación de conocimiento es el proceso de transferencia de conocimiento de un modelo grande a otro más pequeño. Aunque los modelos grandes (como BERT) tienen mayor capacidad de aprendizaje que los modelos pequeños, es posible que esta capacidad no se utilice plenamente. Evaluar un modelo puede ser igual de costoso desde el punto de vista computacional aunque utilice muy poco de su capacidad de conocimiento. La destilación de conocimiento transfiere conocimientos de un modelo grande a otro más pequeño sin pérdida de validez. Como los modelos más pequeños son menos costosos de evaluar, se pueden implantar en hardware menos potente (como un dispositivo móvil, o en este caso, un entorno de Colab)

<center><img src="https://drive.google.com/uc?export=view&id=15uxvIrnqiJJmkqhet_eLGud7CKlQ-145" alt ="DistilBERT" width="80%" /></center>

DistilBERT tiene un 40% menos de parámetros que el modelo BERT, se ejecuta un 60% más rápido y conserva más del 95% del rendimiento de BERT. Originalmente se propuso en el artículo: [_DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter_](https://arxiv.org/abs/1910.01108). Para nuestra fortuna, el modelo se encuentra implementado y disponible en HuggingFace: https://huggingface.co/docs/transformers/model_doc/distilbert.

Vamos entonces a aplicar lo aprendido en la unidad. Recordemos cuál es el pipeline estandar de ejecución de un Transformer:

<center><img src="https://drive.google.com/uc?export=view&id=1yaxcm5ceXkGuMZ5u6iQaZ0sGyYXGU3Pu" alt ="Gráfico ilustrativo de la pipeline de ejecución" width="80%" /></center>



> **La tarea es incremental, por lo tanto es recomendable resolver los puntos en orden.**

# **1. Tokenización**
---
Aplique el preprocesamiento requerido para usar una arquitectura DistilBERT sobre los conjuntos definidos de entrenamiento, validación y prueba. Complete la función `tokenize`, que recibe como argumentos un conjnuto de datos y los parámetros necesarios para tokenizarlos. La función debe retornar un objeto tipo `BatchEncoding` como lo vimos en el taller guiado. Utilice `AutoTokenizer` y la tokenización del modelo `"distilbert-base-uncased"`.

**Entradas**:

* **`model_name`**: un `str` que representa el nombre del tokenizer del modelo pre-entrenado a definir.
* **`X`**, `list`, una lista de secuencuas de texto.
* **`truncate`**, `boolean`, variable booleana para definir si los textos se truncan o no.  
* **`padd`**, `boolean`, variable booleana para definir si los textos se rellenan o no.
* **`tensor`**, un `str` que puede ser `np`, `tf` o `pt` para indicar el tipo de tensor que debe devolver el tokenizador.

**Salida**:

* **`encodings`**: un objeto tipo `BatchEncoding` para tokenizar textos para `DistilBERT`.

In [15]:
# FUNCION CALIFICADA tokenize
from transformers import AutoTokenizer
def tokenize(model_name, X, truncate, padd, tensor):

    padding=True if padd else False

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    encodings = tokenizer(
        X,
        truncation=truncate,
        padding=padd,
        return_tensors=tensor,
        max_length=128)

    return encodings

In [16]:
# TEST_CELL
train_encodings = tokenize(model_name='distilbert-base-uncased',
                           X = X_train,
                           truncate=True,
                           padd=True,
                           tensor= "np")
val_encodings = tokenize(model_name='distilbert-base-uncased',
                         X = X_val,
                         truncate=True,
                         padd=True,
                         tensor= "np")
test_encodings = tokenize(model_name='distilbert-base-uncased',
                          X = X_test,
                          truncate=True,
                          padd=True,
                          tensor= "np")

print("Encoding tipo:", type(train_encodings))
print("Primeros cinco tokens de test:", test_encodings['input_ids'][0][:5])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Encoding tipo: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Primeros cinco tokens de test: [ 101 5712 2022 4974 2075]


**Salida esperada:**
```
Encoding tipo: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Primeros cinco tokens de test: [ 101 5712 2022 4974 2075]
```

### **Evaluar código**

In [18]:
grader.run_test("Test 1_1_1", globals())

Test 1_1_1


In [17]:
grader.run_test("Test 1_1_2", globals())

Test 1_1_2


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# **2. _Fine Tuning_ con _DistilBERT_**
---
Vamos a llevar a cabo el flujo de trabajo neesario para hacer _Fine Tuning_ con un modelo preentrenado.

## **2.1 Definir el modelo**
---
Primero vamos a definir el modelo. Implemente una función llamada `pretrained_model` que reciba como entrada un nombre de modelo y retorne una instancia de ese modelo pre-entrenado utilizando la biblioteca Transformers de HuggingFace. La función debe utilizar `TFAutoModelForSequenceClassification` para cargar el modelo y devolverlo.

**Entradas**:

* **`model_name`**: un `str` que representa el nombre del modelo pre-entrenado a cargar.

**Salida**:

* **`model`**: una instancia del modelo pre-entrenado cargado utilizando `TFAutoModelForSequenceClassification`.

In [21]:
# FUNCION CALIFICADA pretrained_model
from transformers import TFAutoModelForSequenceClassification
def pretrained_model(model_name):
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
    return model

In [22]:
# TEST_CELL
model = pretrained_model('distilbert-base-uncased')
print(type(model))
print(model.summary())

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

<class 'transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification'>
Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
__________

**Salida esperada:**
```
<class 'transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification'>
Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
=================================================================
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None
```

### **Nota importante**
---
Veamos detalladamente la arquitectura del modelo anterior. Tenemos 66,955,010 de parámetros repartidos en cuatro bloques. El primer bloque es el modelo base `distilbert`. Los parámetros de ese bloque (66,362,880) son los que se deben congelar al momento de hacer el calentamiento o _warming up_.

### **Evaluar código**

In [23]:
grader.run_test("Test 2_1_1", globals())

Test 2_1_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [24]:
grader.run_test("Test 2_1_2", globals())

Test 2_1_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

## **2.2 Compilar**True========
---

Igual que con BERT, DistilBERT se define por defecto con dos neuronas de salida con activación linea. Complete entonces la función `compile` que prepara el modelo para el entrenamiento. La función debe recibir como entrada un modelo, un optimizador y una tasa de aprendizaje, y luego compila el modelo con una función de pérdida `CategoricalCrossentropy`kjkjk*

*    **`model`**: una instancia del modelo a entrenar tipo `TFAutoModelForSequenceClassification`.
*    **`optimizer`**: una instancia del optimizador de tipo `keras.optimizers` a utilizar durante el entrenamiento.
*    **`l_r`**: `float`, un número flotante que representa la tasa de aprendizaje.

**Salida:**

* **`model`**: una instancia compilada del modelo tipo `TFAutoModelForSequenceClassification`.

> **Notas:**
  * Utilice el método `.assign()` para ajustar la tasa de aprendizaje del optimizador al valor proporcionado.
  * Recuerde definir `from_logits=True` dentro de la función dlklklklklklklkl8898989898989898989898hhjhjhjhjhjjhjhjhhhhhr.

In [25]:
# FUNCIÓN CALIFICADA compile_model

def compile_model(model, optimizer, l_r):
    # Asigna la tasa de aprendizaje al optimizador
    optimizer.learning_rate.assign(l_r)

    # Define la función de pérdida
    # Importa CategoricalCrossentropy desde tensorflow.keras.losses
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compila el modelo con el optimizador y la función de pérdida
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [26]:
# TEST_CELL
model = compile_model(model=model,
                   optimizer=tf.keras.optimizers.Adam(),
                   l_r=1e-3
                   )
model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': True,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'module': 'keras.losses',
  'class_name': 'CategoricalCrossentropy',
  'config': {'reduction': 'auto',
   'name': 'categorical_crossentropy',
   'from_logits': True,
   'label_smoothing': 0.0,
   'axis': -1,
   'fn': 'categorical_crossentropy'},
  'registered_name': None},
 'metrics': ['accuracy'],
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

**Salida esperada:**
```
{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': True,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'module': 'keras.losses',
  'class_name': 'CategoricalCrossentropy',
  'config': {'reduction': 'auto',
   'name': 'categorical_crossentropy',
   'from_logits': True,
   'label_smoothing': 0.0,
   'axis': -1},
  'registered_name': None},
 'metrics': None,
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}
```

### **Evaluar código**

In [27]:
grader.run_test("Test 2_2_1", globals())

Test 2_2_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [28]:
grader.run_test("Test 2_2_2", globals())

Test 2_2_2


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **2.3 Función de entrenamiento**
---
Complete la función llamada `train_model` que entrena un modelo utilizando un conjunto de datos de entrenamiento. La función debe recibir como un modelo, los datos de entrenamiento, el número de épocas y el tamaño del _batch, y devolver el modelo entrenado.

**Entradas**:

*    **`model`**: una instancia del modelo a entrenar tipo `TFAutoModelForSequenceClassification`
*    **`X_train`**: un objeto tipo `BatchEncoding` para tokenizar textos.
*    **`y_train`**: un arreglo de numpy `np.array` que representa las etiquetas de los datos de entrenamiento.
*    **`X_val`**: un objeto tipo `BatchEncoding` para tokenizar textos.
*    **`y_val`**: un arreglo de numpy `np.array` que representa las etiquetas de los datos de validación.
*    **`epochs`**: `int`, un número entero que representa el número de épocas de entrenamiento.
*    **`batch_size`**: `int`, un número entero que representa el tamaño del _batch_.
*    **`train_base`**: `boolean`, una variable booleana para definir si se congelan o no las capas del modelo base (dependiendo si se quiere hacer _warming up_ o _fine tuning_).

**Salida**:

*  **`history`**: un objeto tipo `History` de tensorflow con la información del entrenamiento del modelo.
*    **`model`**: una instancia del modelo entrenado tipo `TFAutoModelForSequenceClassification`

Para implementar la función, utilice un ciclo `for` para controlar el atributo `trainable` de las capas del modelo, excepto la capa de salida.

In [29]:
# FUNCION CALIFICADA train_model
def train_model(model,
                    X_train, y_train,
                    X_val, y_val,
                    epochs,
                    batch_size,
                    train_base):
    # 1. Controlar si las capas base están entrenables o no
    #for layer in model.layers:
    #    if layer.name != "classifier":  # "classifier" es la capa de salida #que debe entrenarse
    #        layer.trainable = train_base

    # 1. Congelar o descongelar la capa base (`distilbert`)
    for layer in model.layers:
        if "distilbert" in layer.name:  # Congelar sólo la capa base
            layer.trainable = train_base
        else:
            layer.trainable = True  # Las otras capas siempre deben entrenarse
    # 2. Definir Stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    #stopping = tf.keras.callbacks.EarlyStopping(
    #            monitor="val_loss",
    #            patience=5,
    #            mode="min",
    #            restore_best_weights=True
    #        )

    # 3. Entrenar el modelo
    history = model.fit(
        x=X_train.data,       # Datos de entrada tokenizados de entrenamiento
        y=y_train,            # Etiquetas de entrenamiento
        validation_data=(X_val.data, y_val),  # Datos de validación y etiquetas
        validation_split = 0.2,
        epochs=epochs,
        batch_size=batch_size,
        #callbacks=[stopping]
        callbacks=[early_stopping]
    )


    return history, model

In [30]:
# TEST_CELL

history, model = train_model(model=model,
                             X_train=train_encodings,
                             y_train=y_train,
                             X_val=val_encodings,
                             y_val=y_val,
                             epochs=1,
                             batch_size=32,
                             train_base = False)
print(model.summary())
print(history.history.keys())
print('El modelo se ha entrenado durante',len(history.history['val_loss']),'epoch')

501/501 [==============================] - 146s 235ms/step - loss: 0.6940 - accuracy: 0.5573 - val_loss: 0.6859 - val_accuracy: 0.5610
Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 592130 (2.26 MB)
Non-trainable par

**Salida esperada:**
```
Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
=================================================================
Total params: 66,955,010
Trainable params: 592,130
Non-trainable params: 66,362,880
_________________________________________________________________
None
dict_keys(['loss', 'val_loss'])
El modelo se ha entrenado durante 1 epoch
```

### **Evaluar código**

> Tiempo estimado: 2:10:00 h sin GPU, 02:35 m con GPU

In [31]:
grader.run_test("Test 2_3_1", globals())

Test 2_3_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

## **2.4 _Fine Tuning_**
---
Ahora vamos a usar las funciones que definió anteriormente para hacer todo el proceso de ajuste fino, comenzando por hacer un calentamiento o _warming up_ entranando solo la capa final del modelo, y luego entrenando el modelo completo.

Complete la función `fine_tuning`. Esta función recible los datos de entrenamiento y valdación, la tasa de aprendizaje del _warming up_ y la tasa de aprendizaje del _fine tuning_, así como las epochs dedicadas a cada etapa del entrenamiento. La función debe crear un modelo, compilarlo para _warming up_, realizar el _warming up_, y luego compilar el modelo de nuevo para _fine tuning_ (liberando los pesos de todas las capas), y entrenar de nuevo.

**Entrada**:

*    **`X_train`**: un objeto tipo `BatchEncoding` para tokenizar textos.
*    **`y_train`**: un arreglo de numpy `np.array` que representa las etiquetas de los datos de entrenamiento.
*    **`X_val`**: un objeto tipo `BatchEncoding` para tokenizar textos.
*    **`y_val`**: un arreglo de numpy `np.array` que representa las etiquetas de los datos de validación.
*    **`l_r_warming_up`**: `float`, la tasa de aprendizaje a usar durante el calentamiento.
*    **`epochs_warming_up`**: `int`, un número entero que representa el número de épocas de entrenamiento usadas en el calentamiento.
*    **`l_r_fine_tuning`**: `float`, la tasa de aprendizaje a usar durante el _fine tuning_.
*    **`epochs_fine_tuning`**: `int`, un número entero que representa el número de épocas de entrenamiento usadas en el _fine tuning_.

**Salida**:

*  **`history`**: un objeto tipo `History` de tensorflow con la información del entrenamiento del modelo.
*    **`model`**: una instancia del modelo entrenado tipo `TFAutoModelForSequenceClassification`

> **Notas**:
  * Debe definir el modelo usando la función `pretrained_model`, cargando `'distilbert-base-uncased'`.
  * Debe compilar el modelo dos veces usando la función `compile_model`. La primera vez será para configurar el modelo para el _warming up_. La seguda vez que compile será después de haber hecho el _warming up_, para habilitar el entrenamiento de todas las capas y cambiar la tasa de aprendizaje.
  * Analogamente, debe usar dos veces la función `train_model`, una vez para _warming up_, y la segunda vez para _fine tuning_.
  * `tf.keras.optimizers.Adam()` debe ser el optimizador de todos los entrenamientos.
  * La función debe usar un `batch_size` de 32 en todos los entrenamientos.

In [37]:
# FUNCION CALIFICADA fine_tuning
def fine_tuning(X_train, y_train,
                    X_val, y_val,
                    l_r_warming_up,
                    epochs_warming_up,
                    l_r_fine_tuning,
                    epochs_fine_tuning):

    # 1. Crear el modelo preentrenado
    model = pretrained_model("distilbert-base-uncased") #, num_labels=2)

    # 2. Compilar el modelo para warming up
    optimizer = tf.keras.optimizers.Adam(learning_rate=l_r_warming_up)
    model = compile_model(model, optimizer, l_r=l_r_warming_up)

    # Paso 3: Entrenar el modelo en la fase de warming up
    history_warmup, model = train_model(
        model=model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        epochs=epochs_warming_up,
        batch_size=32,
        train_base=False  # Congelar capas base
    )

    # 4. Compilar de nuevo el modelo para fine tuning (liberando las capas base)
    model = compile_model(model, optimizer, l_r=l_r_fine_tuning)

    # 5. Fine tuning: Entrenar liberando todas las capas
    history_fine_tuning, model = train_model(
        model=model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        epochs=epochs_fine_tuning,
        batch_size=32,
        train_base=True  # Liberar todas las capas para el fine tuning
    )


    # Concatenar los historiales de entrenamiento
    #combined_history = {}
    #for key in history_warmup.history.keys():
    #    combined_history[key] = history_warmup.history[key] + history_fine_tuning.history[key]

    # Paso 6: Combinar los historiales de entrenamiento en un solo objeto History

    # Combinar historiales en un solo objeto
    history = history_warmup  # Inicializamos con el historial del warming up
    for key in history.history.keys():
        history.history[key].extend(history_fine_tuning.history[key])  # Añadir registros del fine tuning


    return history, model

In [38]:
# TEST_CELL
history, model = fine_tuning(X_train=train_encodings,
                              y_train=np.asarray(y_train),
                              X_val=val_encodings,
                              y_val=np.asarray(y_val),
                              l_r_warming_up=1e-3,
                              epochs_warming_up=1,
                              l_r_fine_tuning=5e-5,
                              epochs_fine_tuning=1)
print(model.summary())
print('learnign_rate =',model.get_compile_config()['optimizer']['config']['learning_rate'])
print(history.history.keys())
print('El modelo se ha entrenado durante',len(history.history['val_loss']),'epoch')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

501/501 [==============================] - 59s 105ms/step - loss: 0.6868 - accuracy: 0.5611 - val_loss: 0.6858 - val_accuracy: 0.5610
Model: "tf_distil_bert_for_sequence_classification_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_295 (Dropout)       multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainab

**Salida esperada:**
> **Nota**: los valores de `loss` y `val_loss` pueden cambiar.

```
501/501 [==============================] - 147s 226ms/step - loss: 0.6918 - val_loss: 0.6861
501/501 [==============================] - 54s 95ms/step - loss: 0.6859 - val_loss: 0.6859
Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
=================================================================
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None
learnign_rate = 4.999999873689376e-05
dict_keys(['loss', 'val_loss'])
El modelo se ha entrenado durante 1 epoch

```

### **Evaluar código**

In [39]:
## TEACHEAR
history.history

{'loss': [0.6955420970916748, 0.6868057250976562],
 'accuracy': [0.5551326274871826, 0.5610608458518982],
 'val_loss': [0.6865442991256714, 0.6857529878616333],
 'val_accuracy': [0.561025857925415, 0.561025857925415]}

> Tiempo estimado: 4:10:00 h sin GPU, 04:00 m con GPU

In [40]:
grader.run_test("Test 2_4_1", globals())

Test 2_4_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

## **2.5 Evaluar**
---
Complete la función `evaluate_model` que recible un modelo entrenado y lo evalua en un conjunto de prueba.

**Entrada**:

*  **`model`**: una instancia del modelo entrenado tipo `TFAutoModelForSequenceClassification`
*    **`X_test`**: un objeto tipo `BatchEncoding` para tokenizar textos.
*    **`y_test`**: un arreglo de numpy `np.array` que representa las etiquetas de los datos de entrenamiento.

**Salida**:
* **`report`**: un `str` dado por la función `classification_report` de Scikit-Learn.

> Nota: recuerde aplicar `tf.math.softmax` a las predicciones del modelo.

In [41]:
# FUNCIÓN CALIFICADA evaluate
from sklearn.metrics import *
def evaluate(model, X_test, y_test):
    # 1. Realizar predicciones
    raw_predictions = model.predict(X_test.data).logits

    # 2. Aplicar softmax a las predicciones para obtener probabilidades
    probabilities = tf.math.softmax(raw_predictions, axis=-1)

    # 3. Obtener las clases predichas
    y_pred = np.argmax(probabilities, axis=1)

    # 4. Generar el informe de clasificación
    report = classification_report(y_test, y_pred)

    return report

In [42]:
# TEST_CELL
print(evaluate(model=model,
               X_test=test_encodings,
               y_test=y_test))

167/167 [==============================] - 12s 63ms/step
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      2997
           1       0.00      0.00      0.00      2345

    accuracy                           0.56      5342
   macro avg       0.28      0.50      0.36      5342
weighted avg       0.31      0.56      0.40      5342



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Salida esperada**:

```
167/167 [==============================] - 13s 65ms/step
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      2997
           1       0.00      0.00      0.00      2345

    accuracy                           0.56      5342
   macro avg       0.28      0.50      0.36      5342
weighted avg       0.31      0.56      0.40      5342
```

### **Evaluar código**

> Tiempo estimado: 1:00 m sin GPU, 00:29 m con GPU

In [43]:
grader.run_test("Test 2_5_1", globals())

Test 2_5_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

## **Entrenando durante más _epochs_**
----
En este punto usted ha debido programar con éxito todos los pasos necesairios para hacer _Fine Tuning_ con _DistilBERT_ en usando _HuggingFace_, pero no hemos entrenado lo suficiente el modelo. A continuación use su código para el entrenar el modelo durante algunas epochs de calentamiento y otras más de _fine tuning_, cambiando en cada caso las tasas de aprendizaje y evaluando los resultados.

> **Nota: este código no es calificable.**

In [ ]:
history, model = fine_tuning(X_train=train_encodings,
                              y_train=np.asarray(y_train),
                              X_val=val_encodings,
                              y_val=np.asarray(y_val),
                              l_r_warming_up=5e-3,
                              epochs_warming_up=5,
                              l_r_fine_tuning=5e-5,
                              epochs_fine_tuning=20)
print(evaluate(model=model,
               X_test=test_encodings,
               y_test=np.array(y_test)))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
501/501 [==============================] - 132s 231ms/step - loss: 0.7044 - accuracy: 0.5573 - val_loss: 0.6857 - val_accuracy: 0.5610
Epoch 2/5
501/501 [==============================] - 112s 224ms/step - loss: 0.6859 - accuracy: 0.5611 - val_loss: 0.6857 - val_accuracy: 0.5610
Epoch 3/5
501/501 [==============================] - 112s 224ms/step - loss: 0.6857 - accuracy: 0.5611 - val_loss: 0.6857 - val_accuracy: 0.5610
Epoch 4/5
501/501 [==============================] - 109s 218ms/step - loss: 0.6860 - accuracy: 0.5611 - val_loss: 0.6857 - val_accuracy: 0.5610
Epoch 1/20
501/501 [==============================] - 60s 108ms/step - loss: 0.6857 - accuracy: 0.5611 - val_loss: 0.6857 - val_accuracy: 0.5610
Epoch 2/20
501/501 [==============================] - ETA: 0s - loss: 0.6857 - accuracy: 0.5611

KeyboardInterrupt: 

# **Evaluación**


> Tiempo estimado: 06:00:00 h sin GPU, 10:30 m con GPU

In [44]:
grader.submit_task(globals())

Test 1_1_1
Test 1_1_2
Test 2_1_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Test 2_1_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Test 2_2_1


...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Test 2_4_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Test 2_5_1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

# **Referencias**
---
* [*Fine-tuning with custom datasets*](https://huggingface.co/transformers/v3.2.0/custom_datasets.html)

* [*Transformer Models For Custom Text Classification Through Fine-Tuning*](https://towardsdatascience.com/transformer-models-for-custom-text-classification-through-fine-tuning-3b065cc08da1)

* [*Fine-Tuning Hugging Face Model with Custom Dataset*](https://towardsdatascience.com/fine-tuning-hugging-face-model-with-custom-dataset-82b8092f5333)

* [*Fine-Tuning Bert for Tweets Classification ft. Hugging Face*](https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf)

* [*Coronavirus tweets NLP - Text Classification*](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification?resource=download&select=Corona_NLP_test.csv)

* [*covid_tweet_classification.ipynb*](https://colab.research.google.com/github/codistro/Articles/blob/main/covid_tweet_classification.ipynb#scrollTo=bmeDURiI7Q6M)


* _Origen de imágenes_

  - Swatimeena. (2021, 16 diciembre). DistilBERT Text classification using Keras - Swatimeena - Medium. Medium. [Imagen] https://miro.medium.com/v2/resize:fit:720/format:webp/1*e5G4Vdt6gSFkRjs3fD36Pg.png

  - Davaadorj,M. (2022, 19 marzo).
Huggingface-course: Documentation-images - Full Nlp Pipeline [Imagen] https://huggingface.co/datasets/huggingface-course/documentation-images/blob/main/en/chapter2/full_nlp_pipeline.svg


# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*